In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [2]:
from tensorflow.keras.preprocessing.image import img_to_array

In [3]:
from tensorflow.keras.models import load_model

In [4]:
import numpy as np

In [5]:
import cv2

In [6]:
from os.path import dirname, join


In [7]:
prototxtPath = r"C:\Users\ADMIN\Downloads\face mask\face detection\deploy.prototxt"
weightsPath = r"C:\Users\ADMIN\Downloads\face mask\face detection\res10_300x300_ssd_iter_140000.caffemodel"

In [8]:
model=load_model(r'C:\Users\ADMIN\Downloads\face mask\mask.detect.model')

In [9]:
protoPath = join(r'C:\Users\ADMIN\Downloads\face mask\face_d', "deploy.prototxt")
modelPath = join(r'C:\Users\ADMIN\Downloads\face mask\face_d', "res10_300x300_ssd_iter_140000 (1).caffemodel")

In [10]:
net=cv2.dnn.readNet(protoPath,modelPath)

In [11]:
image=cv2.imread(r'C:\Users\ADMIN\Downloads\face mask\example\kkk.png')

In [12]:
image

array([[[186, 183, 145],
        [186, 184, 144],
        [186, 184, 144],
        ...,
        [223, 220, 215],
        [223, 220, 215],
        [223, 220, 215]],

       [[184, 183, 145],
        [184, 183, 145],
        [184, 183, 145],
        ...,
        [223, 220, 215],
        [223, 220, 215],
        [223, 220, 215]],

       [[182, 181, 146],
        [182, 181, 146],
        [183, 182, 144],
        ...,
        [223, 220, 215],
        [223, 220, 215],
        [223, 220, 215]],

       ...,

       [[ 90,  87,  77],
        [ 90,  87,  77],
        [ 91,  88,  78],
        ...,
        [106, 102,  96],
        [106, 102,  96],
        [106, 102,  96]],

       [[ 90,  87,  77],
        [ 91,  88,  78],
        [ 91,  88,  78],
        ...,
        [105, 101,  95],
        [105, 101,  95],
        [105, 101,  95]],

       [[ 90,  87,  77],
        [ 91,  88,  78],
        [ 91,  88,  78],
        ...,
        [104, 100,  94],
        [104, 100,  94],
        [104, 100,  94]]

In [13]:
(h,w)=image.shape[:2]

In [14]:
(h,w)


(500, 600)

In [15]:
blob=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))

In [16]:
blob

array([[[[ 81.,  81.,  76., ..., 119., 119., 119.],
         [ 78.,  79.,  81., ..., 119., 119., 119.],
         [ 68.,  70.,  75., ..., 118., 118., 118.],
         ...,
         [-13., -13., -11., ...,  -1.,   1.,   2.],
         [-14., -13., -15., ...,   0.,   2.,   2.],
         [-14., -13., -14., ...,  -2.,   0.,   0.]],

        [[  6.,   7.,   1., ...,  43.,  43.,  43.],
         [  4.,   5.,   6., ...,  43.,  43.,  43.],
         [ -5.,  -2.,   0., ...,  42.,  42.,  42.],
         ...,
         [-89., -89., -86., ..., -76., -76., -74.],
         [-90., -89., -90., ..., -74., -75., -75.],
         [-90., -89., -90., ..., -76., -77., -77.]],

        [[ 22.,  21.,  13., ...,  92.,  92.,  92.],
         [ 23.,  21.,  20., ...,  92.,  92.,  92.],
         [ 18.,  19.,  16., ...,  91.,  91.,  91.],
         ...,
         [-45., -45., -42., ..., -33., -28., -26.],
         [-46., -45., -46., ..., -33., -27., -27.],
         [-46., -45., -45., ..., -35., -29., -29.]]]], dtype=float32)

In [17]:
blob.shape

(1, 3, 300, 300)

In [18]:
net.setInput(blob)
detections=net.forward() 

In [19]:
detections

array([[[[0.        , 1.        , 0.9984427 , ..., 0.12488029,
          0.6709176 , 0.3542412 ],
         [0.        , 1.        , 0.12920393, ..., 3.9990287 ,
          4.8382664 , 4.984081  ],
         [0.        , 1.        , 0.12446587, ..., 3.994579  ,
          0.85450625, 4.978922  ],
         ...,
         [0.        , 1.        , 0.0841964 , ..., 0.4145316 ,
          0.17800367, 0.51173985],
         [0.        , 1.        , 0.08415166, ..., 0.08340292,
          0.53142047, 0.21906102],
         [0.        , 1.        , 0.08414721, ..., 0.839233  ,
          0.8093825 , 0.95520073]]]], dtype=float32)

#IMP PART 

In [ ]:

for i in range(0,detections.shape[2]):
    confidence=detections[0,0,i,2]
    
    
    if confidence>0.5:
        
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY)=box.astype('int')
        

        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))
        
        
        
        face=image[startY:endY, startX:endX]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        
        (mask,withoutMask)=model.predict(face)[0]
        
        
        label='Mask' if mask>withoutMask else 'No Mask'
        color=(128,0,128) if label=='Mask' else (0,0,255)
        
    
        label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        
    
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
        
        
#for output         
cv2.imshow("Result",image)
#anykey you press it destroy the window
cv2.waitKey(0)
cv2.destroyAllWindows()